In [3]:
import pandas as pd
import math
import csv

In [4]:
arc_df = pd.read_csv('arcGis_data.csv')
print(arc_df.shape)
arc_df.head()

(3764, 11)


,FID,Shape,NUM,GTA06,PD,REGION,GTA01,AREA_M,AREA_H,UTMX_CENT,UTMY_CENT
0,2151,Polygon,1,1,1,1,359,0,0,0,0
1,1826,Polygon,2,2,1,1,360,0,0,0,0
2,2153,Polygon,3,3,1,1,359,0,0,0,0
3,1961,Polygon,4,4,1,1,478,0,0,0,0
4,1960,Polygon,5,5,1,1,478,0,0,0,0


In [5]:
home_df = arc_df[['GTA06']]
home_df = home_df.rename(columns = {'GTA06':'HomeZone'})
home_df.head()

,HomeZone
0,1
1,2
2,3
3,4
4,5


In [9]:
hh_df = pd.read_csv('../GitHub_PORPOS/PORPOS-TMG/Data/SMTO_2015/SMTO_2015_Households.csv')
ps_df = pd.read_csv('../GitHub_PORPOS/PORPOS-TMG/Data/SMTO_2015/SMTO_2015_Respondents.csv')
campus_info = pd.read_csv('../GitHub_PORPOS/PORPOS-TMG/Data/SMTO_2015/Campus_Info.csv')

# List of campus' TTS zones from Joven's MOE data
campus_zones = list(campus_info['Zone'])
school_codes = list(campus_info['Code'])

# List of University Information from OUAC
uni_info = pd.read_csv('../GitHub_PORPOS/PORPOS-TMG/Data/Uni_Info.csv')
uni_info = uni_info.set_index('University')

# Load relevant columns
df = ps_df[['pscampusattend', 'personstatusgrad', 'personstatustime','psmainmodefalltypical', 'pscmpgender','psdrivinglicenseownerflag','psworknumhoursperweek','psage','psexpfactor']]
df = df.join(hh_df[['HmTTS2006', 'hhlivingsituation', 'hhcarnumber','hhnumyoungerthan18','hhnumolderorequalto18','hhincomelevel']])

df = df.rename(columns={'HmTTS2006': 'HomeZone', 'pscampusattend': 'Campus', 'hhlivingsituation': 'Family', 'psmainmodefalltypical': 'Mode_Actual', 
                       'personstatusgrad': 'Level', 'personstatustime': 'Status', 'psuniversityinvolvednumyears': 'Years', 'hhcarnumber': 'Cars',
                       'hhincomelevel': 'Income', 'pscmpgender': 'Gender', 'psdrivinglicenseownerflag': 'Licence', 'psworknumhoursperweek': 'Work',
                       'hhnumyoungerthan18': 'Children', 'hhnumolderorequalto18':'Adults', 'psage': 'Age','psexpfactor': 'Exp_SMTO'})


mode_name_to_num = {"Car - Driver alone": "Auto", "Car - Driver with passenger(s)": "Auto", "Car - Passenger": "Auto", "Taxi": "Auto", 
                    "Transit Bus": "Transit", "Streetcar": "Transit", "Subway/RT": "Transit", "GO Bus": "Transit", "GO Train": "Transit", 
                    "Walk": "Active", "Bicycle": "Active"}
print(df.shape)

# Make Aggregate Mode Column
df['Mode'] = df['Mode_Actual'].replace(mode_name_to_num)
df = df[(df['Mode'] == "Auto") | (df['Mode'] == "Transit") | (df['Mode'] == "Active")]
print(df.shape)

df['Work'].fillna('Unknown', inplace=True) # Fill 11,000 Work NaNs with "Unknowns"
df = df.dropna() # Remove rows with missing data
print(df.shape)

# Convert Campus column to numerical column
df['CampusZone'] = df["Campus"].apply(lambda x: campus_info.Zone[campus_info['School'] == x].tolist()[0])
print(df.shape)

# Convert columns to numerical
df['HomeZone'] = pd.to_numeric(df['HomeZone'], downcast='signed')

print(df.shape)

# Add column with school codes
df['School_Codes'] = df['Campus'].apply(lambda x: uni_info['Code'][x])

df.head()

(15226, 15)
(15066, 16)
(14839, 16)
(14839, 17)
(14839, 17)


,Campus,Level,Status,Mode_Actual,Gender,Licence,Work,Age,Exp_SMTO,HomeZone,Family,Cars,Children,Adults,Income,Mode,CampusZone,School_Codes
0,Scarborough (UTSC),UG,FT,Transit Bus,Female,0,Unknown,20,9.70,261,Live with family/parents,1,3,4,Unknown,Transit,566,SC
1,Downtown Toronto (St. George),Grad,FT,Walk,Female,1,Unknown,25,5.79,71,Live with partner,0,0,2,"$ 90,000 - 119,999",Active,69,SG
2,Downtown Toronto (St. George),UG,FT,Transit Bus,Female,1,Unknown,23,9.06,3714,Live with family/parents,1,0,4,Unknown,Transit,69,SG
3,Downtown Toronto (St. George),UG,FT,Walk,Male,1,Unknown,20,14.67,74,Live with roommates,0,0,4,Unknown,Active,69,SG
4,Downtown Toronto (St. George),Grad,FT,Walk,Male,1,Unknown,27,9.11,71,Live with partner,0,0,2,"$ 30,000 - 59,999",Active,69,SG


In [10]:
# Column Transformation functions

def inc_transform(x):
    if x == "Less than $ 30,000":
        return "Low"
    elif x == "$ 30,000 - 59,999":
        return "Low"
    elif x == "$ 60,000 - 89,999":
        return "Low"
    elif x == "$ 90,000 - 119,999":
        return "High"
    elif x == "$ 120,000 - 149,999":
        return "High"
    elif x == "$ 150,000 - 179,999":
        return "High"
    elif x == "$ 180,000 - 209,999":
        return "High"
    elif x == "$ 210,000 - 239,999":
        return "High"
    elif x == "$ 240,000 +":
        return "High"
    else:
        return "Unknown"
   
    
def work_transform(x):
    if x == "Yes, I work > 40 hours per week":
        return "FT"
    elif x == "Yes, I work 31-40 hours per week":
        return "FT"
    elif x == "Yes, I work part time (21-30 hours per week)":
        return "PT"
    elif x == "Yes, I work part time (11-20 hours per week)":
        return "PT"
    elif x == "Yes, I work part time (<10 hours per week)":
        return "PT"
    elif x == "No, I don't work":
        return "NW"
    else:
        return "Unknown"
    
def fam_transform(x):
    if x == "Live with family/parents":
        return 1
    else:
        return 0
'''
def car_transform(x):
    if x == 0:
        return 0
    elif x == 1: 
        return 1
    else:
        return 2
'''

'\ndef car_transform(x):\n    if x == 0:\n        return 0\n    elif x == 1: \n        return 1\n    else:\n        return 2\n'

In [11]:
df['Work'] = df["Work"].apply(lambda x: work_transform(x))
df['Family'] = df["Family"].apply(lambda x: fam_transform(x))
#df['Cars'] = df["Cars"].apply(lambda x: car_transform(x))
df['Income'] = df["Income"].apply(lambda x: inc_transform(x))
df.head()

,Campus,Level,Status,Mode_Actual,Gender,Licence,Work,Age,Exp_SMTO,HomeZone,Family,Cars,Children,Adults,Income,Mode,CampusZone,School_Codes
0,Scarborough (UTSC),UG,FT,Transit Bus,Female,0,Unknown,20,9.70,261,1,1,3,4,Unknown,Transit,566,SC
1,Downtown Toronto (St. George),Grad,FT,Walk,Female,1,Unknown,25,5.79,71,0,0,0,2,High,Active,69,SG
2,Downtown Toronto (St. George),UG,FT,Transit Bus,Female,1,Unknown,23,9.06,3714,1,1,0,4,Unknown,Transit,69,SG
3,Downtown Toronto (St. George),UG,FT,Walk,Male,1,Unknown,20,14.67,74,0,0,0,4,Unknown,Active,69,SG
4,Downtown Toronto (St. George),Grad,FT,Walk,Male,1,Unknown,27,9.11,71,0,0,0,2,Low,Active,69,SG


In [12]:
# Remove others for Level & Status
df = df[df['Level'] != 'Other']
df = df[df['Status'] != 'Other']

In [13]:
def row_to_segment(x):
    
    return {((x.Level == 'UG') & (x.Status == 'FT') & x.Family): 1,
            ((x.Level == 'UG') & (x.Status == 'FT') & (not x.Family)): 2,
            ((x.Level == 'UG') & (x.Status == 'PT')): 3,
            ((x.Level == 'Grad') & (x.Status == 'FT') & x.Family): 4,
            ((x.Level == 'Grad') & (x.Status == 'FT') & (not x.Family)): 5,
            ((x.Level == 'Grad') & (x.Status == 'PT')): 6}.get(True, 0)

df['Segment'] = df.apply(row_to_segment, axis=1)

df.head()

,Campus,Level,Status,Mode_Actual,Gender,Licence,Work,Age,Exp_SMTO,HomeZone,Family,Cars,Children,Adults,Income,Mode,CampusZone,School_Codes,Segment
0,Scarborough (UTSC),UG,FT,Transit Bus,Female,0,Unknown,20,9.70,261,1,1,3,4,Unknown,Transit,566,SC,1
1,Downtown Toronto (St. George),Grad,FT,Walk,Female,1,Unknown,25,5.79,71,0,0,0,2,High,Active,69,SG,5
2,Downtown Toronto (St. George),UG,FT,Transit Bus,Female,1,Unknown,23,9.06,3714,1,1,0,4,Unknown,Transit,69,SG,1
3,Downtown Toronto (St. George),UG,FT,Walk,Male,1,Unknown,20,14.67,74,0,0,0,4,Unknown,Active,69,SG,2
4,Downtown Toronto (St. George),Grad,FT,Walk,Male,1,Unknown,27,9.11,71,0,0,0,2,Low,Active,69,SG,5


In [17]:
# Dataframe with walk distances
df_path = pd.read_csv('../GitHub_PORPOS/LoS/Walk_Distances.csv')
origins = list(set(list(df_path['Origin'])))
dists = list(df_path['Data'])

# Dataframe with AutoTravelTimes
#df_att = pd.read_csv('../GitHub_PORPOS/LoS/Auto_Travel_Times.csv')
#AutoTravelTimes = list(df_att['Data'])

# Dataframe with TransitTravelTimes
#df_ttt = pd.read_csv('../GitHub_PORPOS/LoS/Transit_Travel_Times.csv')
#TransitTravelTimes = list(df_ttt['Data'])

# Function for distance lookup
not_found = set()
def find_distance(origin, destination, info_num):
    # mode_num: 0 - to find Walking Distance
    #           1 - to find Walking Time
    #           2 - to find Auto Time  
    #           3 - to find Transit Time
    
    try:
        i = origins.index(origin)
    except ValueError:
        not_found.add(origin)
        return -1
    try:
        j = origins.index(destination)
    except ValueError:
        not_found.add(destination)
        return -1
    
    if info_num == 0:
        return dists[i*2392 + j] / 1000
    elif info_num == 1:
        return dists[i*2392 + j] * 15 / 1000 
    elif info_num == 2:
        return AutoTravelTimes[i*2392 + j]
    elif info_num == 3:
        return TransitTravelTimes[i*2392 + j]
    else:
        print("ERROR: Enter correct info_num!")
        return False

# Load distances into dataframe
for i in range(len(campus_zones)):
    home_df["Dist." + school_codes[i]] = home_df['HomeZone'].apply(lambda x: find_distance(x, campus_zones[i],0))
#for i in range(len(campus_zones)):
#    df["WTT." + school_codes[i]] = df['HomeZone'].apply(lambda x: find_distance(x, campus_zones[i],1))
#for i in range(len(campus_zones)):
#    df["AIVTT." + school_codes[i]] = df['HomeZone'].apply(lambda x: find_distance(x, campus_zones[i],2))
#for i in range(len(campus_zones)):
#    df["TPTT." + school_codes[i]] = df['HomeZone'].apply(lambda x: find_distance(x, campus_zones[i],3))
    
print("# of zones not found:", len(not_found))

home_df.head()

# of zones not found: 1441


,HomeZone,Dist.SG,Dist.SC,Dist.MI,Dist.YK,Dist.YG,Dist.RY,Dist.OC
0,1,8.233101,17.76699,27.43587,24.33012,11.45088,6.407150,7.172228
1,2,7.629588,18.88647,26.70272,23.72661,11.60740,5.773918,6.420753
2,3,6.983658,18.75429,26.05679,23.08068,10.79940,5.127988,5.774823
3,4,6.303398,19.60028,25.34593,22.45217,11.28469,4.417128,5.063963
4,5,5.788057,20.27234,24.83059,21.93682,11.30364,3.901787,4.548622


In [19]:
home_df.to_csv('AllZoneDistances_Input.csv', index = False)

In [18]:
home_df.shape

(3764, 8)

In [122]:
# Load enrollment data
enrollment_df = pd.read_csv('../GitHub_PORPOS/PORPOS-TMG/Data/Enrolment/Joven_Enrollment.csv').set_index('School')
def get_log_enrollment(level, school):
    return math.log1p(enrollment_df.loc[school][level]) # Level: UG, Grad, Total
    
for level in ('Total', 'UG', 'Grad'):
    for i in range(len(campus_zones)):
        df[level + "." + school_codes[i]] = get_log_enrollment(level, school_codes[i])

In [123]:
df.head()

,Campus,Level,Status,Mode_Actual,Gender,Licence,Work,Age,Exp_SMTO,HomeZone,...,UG.YG,UG.RY,UG.OC,Grad.SG,Grad.SC,Grad.MI,Grad.YK,Grad.YG,Grad.RY,Grad.OC
0,Scarborough (UTSC),UG,FT,Transit Bus,Female,0,Unknown,20,9.70,261,...,7.758761,10.166928,8.107419,9.591376,5.624018,6.385194,8.26359,4.762174,7.665285,5.159055
1,Downtown Toronto (St. George),Grad,FT,Walk,Female,1,Unknown,25,5.79,71,...,7.758761,10.166928,8.107419,9.591376,5.624018,6.385194,8.26359,4.762174,7.665285,5.159055
2,Downtown Toronto (St. George),UG,FT,Transit Bus,Female,1,Unknown,23,9.06,3714,...,7.758761,10.166928,8.107419,9.591376,5.624018,6.385194,8.26359,4.762174,7.665285,5.159055
3,Downtown Toronto (St. George),UG,FT,Walk,Male,1,Unknown,20,14.67,74,...,7.758761,10.166928,8.107419,9.591376,5.624018,6.385194,8.26359,4.762174,7.665285,5.159055
4,Downtown Toronto (St. George),Grad,FT,Walk,Male,1,Unknown,27,9.11,71,...,7.758761,10.166928,8.107419,9.591376,5.624018,6.385194,8.26359,4.762174,7.665285,5.159055


In [124]:
# One column per campus tuition
for i in range(len(campus_zones)):
    df["Tuition." + school_codes[i]] = uni_info['Tuition'][i]
    
# One single tuition col
#df['Tuition'] = df['Campus'].apply(lambda x: uni_info.loc[x]['Tuition'])

In [125]:
# Domestic Percentage Column
for i in range(len(campus_zones)):
    df["Domestic." + school_codes[i]] = df['Level'].apply(lambda x: uni_info['Domestic%_' + ('Grad' if (x == 'Grad') else 'UG')][i])

In [126]:
for i in range(len(campus_zones)):
    df["Admission_Avg." + school_codes[i]] = uni_info['Admission'][i]

In [18]:
list(uni_info.index)

['Downtown Toronto (St. George)',
 'Scarborough (UTSC)',
 'Mississauga (UTM)',
 'Keele',
 'Glendon',
 'RyersonU',
 'OCADu']

In [19]:
enrollment_df

,Total,UG,Grad
School,,,
SG,53930,39293,14637
SC,11770,11494,276
MI,13298,12706,592
YK,41142,37263,3879
YG,2457,2341,116
RY,28159,26027,2132
OC,3491,3318,173


In [20]:
exp_factors = {}
schoolnames = list(uni_info.index)
for i in range(len(campus_zones)):
    exp_factors[schoolnames[i]] = [enrollment_df.loc[school_codes[i]]['UG'] / enrollment_df['UG'].sum() / df[df['Level'] == 'UG']['Campus'].value_counts(normalize=True)[i]]
    exp_factors[schoolnames[i]].append(enrollment_df.loc[school_codes[i]]['Grad'] / enrollment_df['Grad'].sum() / df[df['Level'] == 'Grad']['Campus'].value_counts(normalize=True)[i]) 
    exp_factors[schoolnames[i]].append(enrollment_df.loc[school_codes[i]]['Total'] / enrollment_df['Total'].sum() / df[df['Level'] == 'Other']['Campus'].value_counts(normalize=True)[i])
    
def load_exp_factors_segment(x):
    if x.Level == 'Other':
        return x.Exp_SMTO
    return exp_factors[x.Campus][0 if x.Level == 'UG' else 1]

def load_exp_factors_level(x):
    return {x.Level == 'UG': exp_factors[x.Campus][0], x.Level == 'Grad': exp_factors[x.Campus][1]}.get(True, exp_factors[x.Campus][2])

df['Exp_Segment'] = df.apply(load_exp_factors_segment, axis = 1)
df['Exp_Level'] = df.apply(load_exp_factors_level, axis = 1)

IndexError: index out of bounds

In [21]:
df.head()

,Campus,Level,Status,Mode_Actual,Gender,Licence,Work,Age,Exp_SMTO,HomeZone,...,Domestic.YG,Domestic.RY,Domestic.OC,Admission_Avg.SG,Admission_Avg.SC,Admission_Avg.MI,Admission_Avg.YK,Admission_Avg.YG,Admission_Avg.RY,Admission_Avg.OC
0,Scarborough (UTSC),UG,FT,Transit Bus,Female,0,Unknown,20,9.70,261,...,0.8921,0.9673,0.8998,0.893,0.841,0.83,0.817,0.817,0.84,0.824
1,Downtown Toronto (St. George),Grad,FT,Walk,Female,1,Unknown,25,5.79,71,...,0.8077,0.8837,0.6786,0.893,0.841,0.83,0.817,0.817,0.84,0.824
2,Downtown Toronto (St. George),UG,FT,Transit Bus,Female,1,Unknown,23,9.06,3714,...,0.8921,0.9673,0.8998,0.893,0.841,0.83,0.817,0.817,0.84,0.824
3,Downtown Toronto (St. George),UG,FT,Walk,Male,1,Unknown,20,14.67,74,...,0.8921,0.9673,0.8998,0.893,0.841,0.83,0.817,0.817,0.84,0.824
4,Downtown Toronto (St. George),Grad,FT,Walk,Male,1,Unknown,27,9.11,71,...,0.8077,0.8837,0.6786,0.893,0.841,0.83,0.817,0.817,0.84,0.824


In [127]:
# Remove columns we don't need anymore:
del df['CampusZone']
del df['Exp_SMTO']

In [128]:
for i in school_codes:
    df = df[df['WTT.' + i] != -1]
    df = df[df['AIVTT.' + i] < 1000]

In [129]:
df.shape

(14357, 87)

In [19]:
df.columns

Index(['Campus', 'Level', 'Status', 'Mode_Actual', 'Gender', 'Licence', 'Work',
       'Age', 'HomeZone', 'Family', 'Cars', 'Children', 'Adults', 'Income',
       'Mode', 'School_Codes', 'Segment', 'Dist.SG', 'Dist.SC', 'Dist.MI',
       'Dist.YK', 'Dist.YG', 'Dist.RY', 'Dist.OC', 'WTT.SG', 'WTT.SC',
       'WTT.MI', 'WTT.YK', 'WTT.YG', 'WTT.RY', 'WTT.OC', 'AIVTT.SG',
       'AIVTT.SC', 'AIVTT.MI', 'AIVTT.YK', 'AIVTT.YG', 'AIVTT.RY', 'AIVTT.OC',
       'TPTT.SG', 'TPTT.SC', 'TPTT.MI', 'TPTT.YK', 'TPTT.YG', 'TPTT.RY',
       'TPTT.OC', 'Total.SG', 'Total.SC', 'Total.MI', 'Total.YK', 'Total.YG',
       'Total.RY', 'Total.OC', 'UG.SG', 'UG.SC', 'UG.MI', 'UG.YK', 'UG.YG',
       'UG.RY', 'UG.OC', 'Grad.SG', 'Grad.SC', 'Grad.MI', 'Grad.YK', 'Grad.YG',
       'Grad.RY', 'Grad.OC', 'Tuition.SG', 'Tuition.SC', 'Tuition.MI',
       'Tuition.YK', 'Tuition.YG', 'Tuition.RY', 'Tuition.OC', 'Domestic.SG',
       'Domestic.SC', 'Domestic.MI', 'Domestic.YK', 'Domestic.YG',
       'Domestic.RY', 'Dome

In [130]:
df['Segment'].value_counts()

1    6830
2    3433
5    2222
4     846
3     689
6     337
Name: Segment, dtype: int64

In [26]:
#df.to_csv('SMTO_2015_Complete_Input_NoOthers.csv', index = False)

In [53]:
df.head()

,Campus,Level,Status,Mode_Actual,Gender,Licence,Work,Age,HomeZone,Family,...,Domestic.YG,Domestic.RY,Domestic.OC,Admission_Avg.SG,Admission_Avg.SC,Admission_Avg.MI,Admission_Avg.YK,Admission_Avg.YG,Admission_Avg.RY,Admission_Avg.OC
0,Scarborough (UTSC),UG,FT,Transit Bus,Female,0,Unknown,20,261,1,...,0.8921,0.9673,0.8998,0.893,0.841,0.83,0.817,0.817,0.84,0.824
1,Downtown Toronto (St. George),Grad,FT,Walk,Female,1,Unknown,25,71,0,...,0.8077,0.8837,0.6786,0.893,0.841,0.83,0.817,0.817,0.84,0.824
2,Downtown Toronto (St. George),UG,FT,Transit Bus,Female,1,Unknown,23,3714,1,...,0.8921,0.9673,0.8998,0.893,0.841,0.83,0.817,0.817,0.84,0.824
3,Downtown Toronto (St. George),UG,FT,Walk,Male,1,Unknown,20,74,0,...,0.8921,0.9673,0.8998,0.893,0.841,0.83,0.817,0.817,0.84,0.824
4,Downtown Toronto (St. George),Grad,FT,Walk,Male,1,Unknown,27,71,0,...,0.8077,0.8837,0.6786,0.893,0.841,0.83,0.817,0.817,0.84,0.824


In [42]:
pred_df = pd.read_csv('mlogit_prediction.csv')
pred_df.set_index(original_df['Origin'],inplace = True)
pred_df.loc[1]

SG    0.511608
MI    0.013675
OC    0.045430
RY    0.288467
SC    0.048938
YG    0.022725
YK    0.069158
Name: 1, dtype: float64

In [10]:
original_df = pd.read_csv('../Data/Full_Zones_Dists.csv')
original_df

,Origin,Dist.SG,Dist.SC,Dist.MI,Dist.YK,Dist.YG,Dist.RY,Dist.OC
0,1,8.233101,17.76699,27.435870,24.33012,11.45088,6.407150,7.172228
1,2,7.629588,18.88647,26.702720,23.72661,11.60740,5.773918,6.420753
2,3,6.983658,18.75429,26.056790,23.08068,10.79940,5.127988,5.774823
3,4,6.303398,19.60028,25.345930,22.45217,11.28469,4.417128,5.063963
4,5,5.788057,20.27234,24.830590,21.93682,11.30364,3.901787,4.548622
5,6,6.541063,20.54051,25.583600,22.68983,11.57180,4.654793,5.301628
6,7,7.402354,21.28565,26.441880,23.55112,12.43309,5.516084,6.162919
7,8,7.723674,20.48058,26.763200,23.87244,12.16499,5.837404,6.484239
8,9,6.274250,22.89256,25.022400,22.42302,14.04000,4.387980,5.034815
9,10,5.522103,21.88488,24.270250,21.67087,13.03231,3.635833,4.282668


In [11]:
original_df = original_df.set_index(pred_df.index)
original_df['Pred_School'] = pred_df.idxmax(axis = 1)

In [30]:
original_df['Pred_School'].value_counts()

SG    1107
YK     839
SC     441
MI       5
Name: Pred_School, dtype: int64

In [14]:
homezones = set(original_df['Origin'].value_counts().index.tolist())
zone_to_campus = {}

for i in homezones:
    SG = ((original_df['Pred_School'] == 'SG') & (original_df['Origin'] == i)).sum()
    YK = ((original_df['Pred_School'] == 'YK') & (original_df['Origin'] == i)).sum()
    RY = ((original_df['Pred_School'] == 'RY') & (original_df['Origin'] == i)).sum()
    SC = ((original_df['Pred_School'] == 'SC') & (original_df['Origin'] == i)).sum()
    MI = ((original_df['Pred_School'] == 'MI') & (original_df['Origin'] == i)).sum()
    OC = ((original_df['Pred_School'] == 'OC') & (original_df['Origin'] == i)).sum()
    YG = ((original_df['Pred_School'] == 'YG') & (original_df['Origin'] == i)).sum()

    x_names = ['SG','YK','RY','SC','MI','OC','YG']
    x = [SG,YK,RY,SC,MI,OC,YG]
    x_num = [1,2,3,4,5,6,7] 
    
    zone_to_campus[i] = x_names[x.index(max(x))]

In [16]:
top_school_df = original_df.from_dict(zone_to_campus,orient='index')
print(top_school_df.shape)
top_school_df

(2392, 1)


,0
1,SG
2,SG
3,SG
4,SG
5,SG
6,SG
7,SG
8,SG
9,SG
10,SG


In [17]:
top_school_df = top_school_df.rename(columns = {0:'code'})

In [18]:
def code_to_num(x):
    if x == 'SG':
        return 1
    elif x == 'YK':
        return 2
    elif x == 'RY':
        return 3
    elif x == 'SC':
        return 4
    elif x == 'MI':
        return 5
    elif x == 'OC':
        return 6
    elif x == 'YG':
        return 7

top_school_df['num'] = top_school_df['code'].apply(lambda x: code_to_num(x))

In [20]:
top_school_df

,code,num
1,SG,1
2,SG,1
3,SG,1
4,SG,1
5,SG,1
6,SG,1
7,SG,1
8,SG,1
9,SG,1
10,SG,1


In [21]:
arc_df = pd.read_csv('arcGis_data.csv')
print(arc_df.shape)
arc_df.head()

(3764, 11)


,FID,Shape,NUM,GTA06,PD,REGION,GTA01,AREA_M,AREA_H,UTMX_CENT,UTMY_CENT
0,2151,Polygon,1,1,1,1,359,0,0,0,0
1,1826,Polygon,2,2,1,1,360,0,0,0,0
2,2153,Polygon,3,3,1,1,359,0,0,0,0
3,1961,Polygon,4,4,1,1,478,0,0,0,0
4,1960,Polygon,5,5,1,1,478,0,0,0,0


In [22]:
import numpy as np
arc_df['mlogit_School'] = np.nan
arc_df.head()

,FID,Shape,NUM,GTA06,PD,REGION,GTA01,AREA_M,AREA_H,UTMX_CENT,UTMY_CENT,mlogit_School
0,2151,Polygon,1,1,1,1,359,0,0,0,0,NaN
1,1826,Polygon,2,2,1,1,360,0,0,0,0,NaN
2,2153,Polygon,3,3,1,1,359,0,0,0,0,NaN
3,1961,Polygon,4,4,1,1,478,0,0,0,0,NaN
4,1960,Polygon,5,5,1,1,478,0,0,0,0,NaN


In [24]:
my_zones = list(top_school_df.index)
all_zones = list(arc_df['GTA06'])
not_found = set()
for i in range(len(my_zones)):
    try:
        arc_df.at[all_zones.index(my_zones[i]), 'mlogit_School'] = list(top_school_df['num'])[i]  
    except:
        not_found.add(my_zones[i])

In [37]:
not_found

{6000,
 6401,
 6402,
 6403,
 6404,
 6405,
 6406,
 6407,
 6408,
 6409,
 6410,
 8000,
 8500,
 8750,
 8751,
 8752,
 8753,
 8754,
 8755,
 8756,
 8757,
 8758,
 8759,
 8760,
 8761,
 8762,
 8763,
 8764,
 8765,
 8766,
 8767,
 8768,
 8769,
 8770,
 8771,
 8772,
 8773,
 9000,
 9001,
 9002,
 9003,
 9004,
 9005,
 9006,
 9007,
 9008,
 9009,
 9010,
 9011,
 9012,
 9013,
 9014,
 9015,
 9016,
 9250,
 9251,
 9252,
 9253,
 9254,
 9255,
 9256,
 9257,
 9500,
 9501,
 9750,
 9751,
 9763,
 9764,
 9765}

In [36]:
arc_df.head()

,FID,Shape,NUM,GTA06,PD,REGION,GTA01,AREA_M,AREA_H,UTMX_CENT,UTMY_CENT,mlogit_School
0,2151,Polygon,1,1,1,1,359,0,0,0,0,1.0
1,1826,Polygon,2,2,1,1,360,0,0,0,0,1.0
2,2153,Polygon,3,3,1,1,359,0,0,0,0,1.0
3,1961,Polygon,4,4,1,1,478,0,0,0,0,1.0
4,1960,Polygon,5,5,1,1,478,0,0,0,0,1.0


In [28]:
arc_df['mlogit_School'].value_counts()

1.0    1100
2.0     797
4.0     421
5.0       5
Name: mlogit_School, dtype: int64

In [29]:
arc_df.to_csv('mlogitPrediction_col.csv', index = False)